In [82]:
from oddsapi import OddsApiClient
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime as dt
import os
from dotenv import load_dotenv
load_dotenv()

MYDIR = ("odds")







In [33]:
from enum import Enum
class SportsBooks(Enum):
    bet_online = "betonlineag"
    barstool = "barstool"
    betfair = "betfair"
    betmgm =  "betmgm"
    betrivers = "betrivers"
    betus = "betus"
    bovada = "bovada"
    draftkings = "draftkings"
    fanduel = "fanduel"
    foxbet = "foxbet"
    gtbet = "gtbets"
    intertops = "intertops"
    lowwig = "lowvig"
    mybookie = "mybookieag"
    pointsbet = "pointsbetus"
    sugarhouse = "sugarhouse"
    twinspires = "twinspires"
    unibet = "unibet"
    williamhill = "williamhill_us"
    wynnbet = "wynnbet"



str

In [4]:
url = "https://api.the-odds-api.com"
ext = "/v3/sports/"
params = {"apiKey": key}
r = requests.get(url=url + ext, params=params)
r.status_code

200

In [ ]:
r.content
sports = json.loads(r.text)
sports['data'][0]
for i in sports['data']:
    print(i['title'], i['active'])

In [37]:

url = "https://api.the-odds-api.com"
req = f"/v3/odds/"
params = {
    "apiKey": key,
    "sport" :'basketball_nba',
"region" :'us',
'mkt':'h2h',
'oddsFormat': 'american' 
}
odds_req = requests.get(url + req, params=params)

In [38]:
odds = json.loads(odds_req.text)
odds = odds['data']
# odds = odds[0]
# arb = []
# for game in odds:
#     if dt.now() > dt.fromtimestamp(game['commence_time']):
#         continue
#     g = Game(game)
#     if g.arb_exists():
#         print(f"Arb exists for {g.away} @ {g.home} at {g.start_time}! Secure that bag!")
#         arb.append(game)
# print(len(odds))
# for game in arb:
#     print(game.beat_bookies())
odds


[{'id': '3a814a4f0631841d5f7b5a7c62e6dc77',
  'sport_key': 'basketball_nba',
  'sport_nice': 'NBA',
  'teams': ['Boston Celtics', 'Golden State Warriors'],
  'commence_time': 1655427600,
  'home_team': 'Boston Celtics',
  'sites': [{'site_key': 'pointsbetus',
    'site_nice': 'PointsBet (US)',
    'last_update': 1655350662,
    'odds': {'h2h': [-171, 140]}},
   {'site_key': 'draftkings',
    'site_nice': 'DraftKings',
    'last_update': 1655350829,
    'odds': {'h2h': [-170, 150]}},
   {'site_key': 'betmgm',
    'site_nice': 'BetMGM',
    'last_update': 1655350526,
    'odds': {'h2h': [-165, 140]}},
   {'site_key': 'williamhill_us',
    'site_nice': 'William Hill (US)',
    'last_update': 1655350734,
    'odds': {'h2h': [-170, 145]}},
   {'site_key': 'betfair',
    'site_nice': 'Betfair',
    'last_update': 1655350493,
    'odds': {'h2h': [-159, 154], 'h2h_lay': [-154, 160]}},
   {'site_key': 'fanduel',
    'site_nice': 'FanDuel',
    'last_update': 1655350777,
    'odds': {'h2h': [-16

In [40]:
def get_all_odds(sites, home_first = True,draw_possible=False):
    '''
    Given the sites value from an Odds-API request
    return a formatted dict of all of the odds for logging
    Inputs:
        sites: list of odds from various dicts of sports betting odds
        draw_possible: (bool) boolean to denote whether the sport has draws in H2H odds
                        (not currently supported) 
    '''
    odds = {}
    for site in sites:
        name = site['site_key']
        last_update = site['last_update']
        line = site['odds']['h2h']
        if home_first:
            home_odds = line[0]
            away_odds = line[1]
        else:
            home_odds = line[1]
            away_odds = line[0]
        odds[name + '_home'] = home_odds
        odds[name + '_away'] = away_odds
        odds[name + '_last_update'] = dt.fromtimestamp(last_update)
    return odds

get_all_odds(odds[0]['sites'])

{'pointsbetus_home': -171,
 'pointsbetus_away': 140,
 'pointsbetus_last_update': datetime.datetime(2022, 6, 15, 22, 37, 42),
 'draftkings_home': -170,
 'draftkings_away': 150,
 'draftkings_last_update': datetime.datetime(2022, 6, 15, 22, 40, 29),
 'betmgm_home': -165,
 'betmgm_away': 140,
 'betmgm_last_update': datetime.datetime(2022, 6, 15, 22, 35, 26),
 'williamhill_us_home': -170,
 'williamhill_us_away': 145,
 'williamhill_us_last_update': datetime.datetime(2022, 6, 15, 22, 38, 54),
 'betfair_home': -159,
 'betfair_away': 154,
 'betfair_last_update': datetime.datetime(2022, 6, 15, 22, 34, 53),
 'fanduel_home': -168,
 'fanduel_away': 142,
 'fanduel_last_update': datetime.datetime(2022, 6, 15, 22, 39, 37),
 'lowvig_home': -165,
 'lowvig_away': 145,
 'lowvig_last_update': datetime.datetime(2022, 6, 15, 22, 40, 2),
 'unibet_home': -167,
 'unibet_away': 140,
 'unibet_last_update': datetime.datetime(2022, 6, 15, 22, 40, 38),
 'twinspires_home': -167,
 'twinspires_away': 140,
 'twinspires_

In [88]:
class OddsLogger(object):
    """
    Object to automatically odds for backtesting purposes.
    Given a list of games, formats those whose start_time is within an 
    hour of the current time into a .csv file under the appropriate diretory
    current time and 

    Args:
        sport (str): the sport to log the odds on the sportsbooks. Development to come
    """
    def __init__(self, sport):
        self.__api_key = os.getenv('API_KEY')
        base_url = "https://api.the-odds-api.com"
        odds_endpoint = f"/v3/odds/"
        params = {
            "apiKey": self.__api_key,
            "sport" :'baseball_mlb',
            "region" :'us',
            'mkt':'h2h',
            'oddsFormat': 'american'
        }
        odds_req = requests.get(url + req, params=params)
        odds = json.loads(odds_req.text)
        odds = odds['data']
        self.games = []
        for game in odds:
            if dt.fromtimestamp(game['commence_time']) < dt.now(): #Ignore live odds
                continue
            row = pd.Series()
            row["ID"] = game['id']
            row['Sport'] = game['sport_nice']
            row['Home'] = game['home_team']
            row['Away'] = [x for x in game['teams'] if x != game['home_team']][0]
            row['Start Time'] = dt.fromtimestamp(game['commence_time'])
            home_first = True if game['teams'][0] == game['home_team'] else False
            odds_by_sb = get_all_odds(game['sites'], home_first, False)
            books_quoting = [s['site_key'] for s in game['sites']]
            for book in SportsBooks:
                if book.name in books_quoting:
                    row[f'{book.name}_last_update'] = odds_by_sb[f"{book.name}_last_update"]
                    row[f"{book.name}_home"] = odds_by_sb[f"{book.name}_home"]
                    row[f"{book.name}_away"] = odds_by_sb[f"{book.name}_away"]
                else:
                    row[f'{book.name}_last_update'] = np.nan
                    row[f"{book.name}_home"] = np.nan
                    row[f"{book.name}_away"] = np.nan
            self.games.append(row)
        self.odds_frame = pd.DataFrame(self.games).set_index("ID")
        self.odds_by_month = self.split_months()
        self.merge_with_existing_odds()


    def get_all_odds(self, sites, home_first = True,draw_possible=False):
        '''
        Given the sites value from an Odds-API request
        return a formatted dict of all of the odds for logging
        Inputs:
            sites: list of odds from various dicts of sports betting odds
            draw_possible: (bool) boolean to denote whether the sport has draws in H2H odds
                            (not currently supported) 
        '''
        odds = {}
        for site in sites:
            name = site['site_key']
            last_update = site['last_update']
            line = site['odds']['h2h']
            if home_first:
                home_odds = line[0]
                away_odds = line[1]
            else:
                home_odds = line[1]
                away_odds = line[0]
            odds[name + '_home'] = home_odds
            odds[name + '_away'] = away_odds
            odds[name + '_last_update'] = last_update
            if draw_possible:
                odds[name + '_draw'] = line[-1]
        return odds

    def split_months(self):
        g = self.odds_frame.groupby(pd.Grouper(key="Start Time", freq="M"))
        return [group for _,group in g]

    def merge_with_existing_odds(self):
        """
        Iterates through dataframes in self.odds_by_month and merges current
        data with existing data prioritizing the most current lines
        """
        if not os.path.isdir(MYDIR):
            os.makedirs(MYDIR)
        for m in self.odds_by_month:
            month = m['Start Time'][0].strftime("%B")
            year = m['Start Time'][0].year
            path = MYDIR + "\\" + month + "_" + str(year) + ".csv"
            if not os.path.exists(path):
                m.to_csv(path)
            else:
                df = pd.read_csv(path, index_col="ID")
                df = pd.concat([df,m], axis = 0)
                df = df[~df.index.duplicated(keep='last')]
                df.to_csv(path)

        
o = OddsLogger('baseball_mlb')
o.odds_frame
        

<ipython-input-88-dbcf68f239e9>:29: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  row = pd.Series()


,Sport,Home,Away,Start Time,bet_online_last_update,bet_online_home,bet_online_away,barstool_last_update,barstool_home,barstool_away,...,twinspires_away,unibet_last_update,unibet_home,unibet_away,williamhill_last_update,williamhill_home,williamhill_away,wynnbet_last_update,wynnbet_home,wynnbet_away
ID,,,,,,,,,,,,,,,,,,,,,
cfa00da43b70a78968b305e6749988b1,MLB,New York Mets,Miami Marlins,2022-06-20 12:11:00,NaN,NaN,NaN,2022-06-19 22:23:19,-159,133,...,133,2022-06-19 22:27:50,-161,133,NaN,NaN,NaN,2022-06-19 22:23:17,-154,143
6a07f335cc606af5a9e3a2efac17cfe7,MLB,Pittsburgh Pirates,Chicago Cubs,2022-06-20 18:06:00,NaN,NaN,NaN,2022-06-19 22:23:19,-109,-106,...,-106,2022-06-19 22:27:50,-114,-105,NaN,NaN,NaN,2022-06-19 22:23:17,-110,100
d0520126af4d7adf5202d773c5533018,MLB,Boston Red Sox,Detroit Tigers,2022-06-20 18:11:00,NaN,NaN,NaN,2022-06-19 22:23:19,-180,155,...,155,2022-06-19 22:27:50,-189,155,NaN,NaN,NaN,2022-06-19 22:23:17,-175,162
c0b417af29747d68208c31fecd48a28a,MLB,Tampa Bay Rays,New York Yankees,2022-06-20 18:11:00,NaN,NaN,NaN,2022-06-19 22:23:19,108,-125,...,-130,2022-06-19 22:27:50,108,-130,NaN,NaN,NaN,2022-06-19 22:23:17,118,-128
ca481655bb4b70b3f924861eb8846d94,MLB,Atlanta Braves,San Francisco Giants,2022-06-20 18:21:00,NaN,NaN,NaN,2022-06-19 22:23:19,-139,118,...,118,2022-06-19 22:27:50,-139,118,NaN,NaN,NaN,2022-06-19 22:23:17,-147,137
278f6222e7c8c93ded6334b3eed41208,MLB,Milwaukee Brewers,St. Louis Cardinals,2022-06-20 19:11:00,NaN,NaN,NaN,2022-06-19 22:23:19,-159,135,...,135,2022-06-19 22:27:50,-159,135,NaN,NaN,NaN,2022-06-19 22:23:17,-147,136
41c91e58d6d3c4b165c55516d680f76a,MLB,Chicago White Sox,Toronto Blue Jays,2022-06-20 19:11:00,NaN,NaN,NaN,2022-06-19 22:23:19,115,-137,...,-137,2022-06-19 22:27:50,115,-137,NaN,NaN,NaN,2022-06-19 22:23:17,120,-130
b15162d20b618c62b0739a81460b59d7,MLB,Los Angeles Angels,Kansas City Royals,2022-06-20 20:39:00,NaN,NaN,NaN,2022-06-19 22:23:19,-186,155,...,155,2022-06-19 22:27:50,-185,155,NaN,NaN,NaN,2022-06-19 22:23:17,-189,175
a5e970a9baf6b912c6669c15d03163d6,MLB,San Diego Padres,Arizona Diamondbacks,2022-06-20 20:41:00,NaN,NaN,NaN,2022-06-19 22:23:19,-195,165,...,165,2022-06-19 22:27:50,-208,165,NaN,NaN,NaN,2022-06-19 22:23:17,-175,162


In [68]:
pd.DataFrame(o.games).to_csv('test.csv')

In [17]:
from fractions import Fraction
from sympy import symbols, Eq, solve
MY_SPORTSBOOKS = ['Barstool Sportsbook', 'DraftKings', "BetMGM", 'BetRivers', 'FanDuel', 'PointsBet (US)', 'William Hill (Ceasars)']

class Game(object):
    '''
    Object used to organize data from OddsAPI client 
    for processing later
    Inputs:
        data - (dict) data from OddsAPI response. 
        Must include: 'sports_key', 'teams', 'home_team', 
                    'sites'

    '''
    def __init__(self, data):
        self.sport = data['sport_nice']
        self.teams = data['teams']
        self.home = data['home_team']
        for team in self.teams:
            if team != self.home:
                self.away = team
        self.h2h_odds = {}
        for site in data['sites']:
            site_name = site['site_nice']
            self.h2h_odds[site_name] = site['odds']['h2h']
        # self.h2h_odds = pd.DataFrame(self.h2h_odds).T
        # self.h2h_odds.columns = self.teams
        self.start_time = dt.fromtimestamp(int(data['commence_time']))
        self.num_sites = ['sites_count']
        self.best_bettable_odds()
    
    def __repr__(self):
        return f"Game object for {self.away} @ {self.home} on {self.start_time}"

    def best_bettable_odds(self):
        '''
        Iterates through the odds and finds the best odds on both sides 
        Currently only supports games with no draws
        '''
        best_team1 = None
        best_team1_site = None
        best_team2 = None
        best_team2_site = None
        for site, odds in self.h2h_odds.items():
            if site in MY_SPORTSBOOKS:
                if not best_team1:
                    best_team1 = odds[0]
                    best_team1_site = site
                    best_team2 = odds[1]
                    best_team2_site = site
                else:
                    if odds[0] > best_team1:
                        best_team1 = odds[0]
                        best_team1_site = site
                    if odds[1] > best_team2:
                        best_team2 = odds[1]
                        best_team2_site = site
                self.best_odds = {
                    f"{self.teams[0]}": (best_team1, best_team1_site),
                    f"{self.teams[1]}": (best_team2, best_team2_site)
                }
        # print(self.best_odds)

    def arb_exists(self):
        sure_bet = 0
        for odds in self.best_odds.values():
            d_odds = Game.decimal_odds(odds[0])
            sure_bet += 1 / d_odds
        if sure_bet < 1:
            return True
        else:
            return False

    @staticmethod
    def decimal_odds(odds: int) -> float:
        """
        :param odds: Integer (e.g., -350) or String (e.g., '3/1' or '5/4').
        :return: Float. Odds expressed in Decimal terms.
        """
        if isinstance(odds, float):
            return odds

        elif isinstance(odds, int):
            if odds >= 100:
                return abs(1 + (odds / 100))
            elif odds <= -101 :
                return 100 / abs(odds) + 1
            else:
                return float(odds)

        elif "/" in odds:
            odds = Fraction(odds)
            return round((odds.numerator / odds.denominator) + 1, 2)
        
    def beat_bookies(self, total_stake):
        odds1 = self.best_odds[self.teams[0]][0]
        odds2 = self.best_odds[self.teams[1]][0]
        x, y = symbols('x y')
        eq1 = Eq(x + y - total_stake, 0) # total_stake = x + y
        eq2 = Eq((odds2*y) - odds1*x, 0) # odds1*x = odds2*y
        stakes = solve((eq1,eq2), (x, y))
        total_investment = stakes[x] + stakes[y]
        profit1 = odds1*stakes[x] - total_stake
        profit2 = odds2*stakes[y] - total_stake
        print(type(profit1), type(total_investment))
        benefit1 = f'{round(profit1 / total_investment * 100,2)}%'
        benefit2 = f'{round(profit2 / total_investment * 100,2)}%'
        dict_gabmling = {f'{self.teams[0]} stake':stakes[x], f'{self.teams[1]} stake':stakes[y], 'Profit1':profit1, 'Profit2':profit2,
                        'Benefit1': benefit1, 'Benefit2': benefit2}
        return dict_gabmling

        

g = Game(odds)
g.beat_bookies(500)

<class 'sympy.core.numbers.Integer'> <class 'sympy.core.numbers.Integer'>


{'Detroit Tigers stake': 29500,
 'New York Yankees stake': -29000,
 'Profit1': 8554500,
 'Profit2': 8554500,
 'Benefit1': '1710900%',
 'Benefit2': '1710900%'}

In [9]:
g.h2h_odds

{'BetOnline.ag': [241, -270],
 'MyBookie.ag': [245, -305],
 'FOX Bet': [255, -357],
 'DraftKings': [250, -350],
 'WynnBET': [250, -278],
 'GTbets': [267, -363],
 'BetUS': [230, -280],
 'William Hill (US)': [240, -280],
 'Bovada': [260, -375],
 'LowVig.ag': [241, -270],
 'PointsBet (US)': [285, -375],
 'FanDuel': [290, -410],
 'BetMGM': [280, -400],
 'Unibet': [280, -385],
 'TwinSpires': [280, -385],
 'BetRivers': [245, -295],
 'SugarHouse': [280, -385],
 'Barstool Sportsbook': [280, -385],
 'Intertops': [240, -280],
 'Betfair': [168, -588]}

In [99]:
import mlbgame

day = mlbgame.day(2022, 6, 12, home = "Cubs", away="Cubs")
# game = day[0]
# game
day

[]

In [13]:
from pybaseball import schedule_and_record
data = schedule_and_record(2022, 'NYY')
# data.to_csv('test.csv')
data.head()

,Date,Tm,Home_Away,Opp,W/L,R,RA,Inn,W-L,Rank,GB,Win,Loss,Save,Time,D/N,Attendance,cLI,Streak,Orig. Scheduled
1,"Friday, Apr 8",NYY,Home,BOS,W-wo,6.0,5.0,11.0,1-0,1.0,Tied,King,Crawford,None,3:56,D,46097.0,.97,1.0,None
2,"Saturday, Apr 9",NYY,Home,BOS,W,4.0,2.0,9.0,2-0,1.0,Tied,Luetge,Pivetta,Chapman,2:58,D,46882.0,1.03,2.0,None
3,"Sunday, Apr 10",NYY,Home,BOS,L,3.0,4.0,9.0,2-1,2.0,1.0,Crawford,Schmidt,Diekman,3:40,N,40108.0,1.04,-1.0,None
4,"Monday, Apr 11",NYY,Home,TOR,L,0.0,3.0,9.0,2-2,3.0,1.0,Manoah,Taillon,Romano,3:03,N,26211.0,1.05,-2.0,None
5,"Tuesday, Apr 12",NYY,Home,TOR,W,4.0,0.0,9.0,3-2,2.0,1.0,Holmes,Kikuchi,None,3:07,N,25068.0,1.03,1.0,None
